In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Read Data For WA, NY, TX, CA and from 2018 to 2022

In [2]:
df = pd.read_parquet(r"C:\Users\Forcessofnature\Downloads\analyticsDF.parquet")
df

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,total_units_val,debt_to_income_ratio_val,applicant_ethnicitys,co_applicant_ethnicitys,applicant_races,co_applicant_races,denial_reasons
0,2022,5493000YNV8IX4VD3X12,12420,TX,48209,48209010100,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,301,623,50,1,43.00000,Hispanic or Latino,Missing,Missing,Missing,Not applicable
1,2022,5493000YNV8IX4VD3X12,28660,TX,48027,48027022504,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,1619,2445,13,1,33.00000,Missing,Missing,Missing,Missing,Not applicable
2,2022,5493000YNV8IX4VD3X12,43300,TX,48181,48181000304,C,FHA:First Lien,Single Family (1-4 Units):Manufactured,Ethnicity Not Available,...,1015,1990,26,1,None,Missing,Missing,White,White,Collateral
3,2022,5493000YNV8IX4VD3X12,15180,TX,48061,48061013500,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,549,765,58,1,-99.00000,Missing,Missing,Missing,Missing,Not applicable
4,2022,5493000YNV8IX4VD3X12,12420,TX,48491,48491020330,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,1069,1353,19,1,20.00000,Not Hispanic or Latino,Missing,Asian,Missing,Not applicable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9712773,2020,549300VZVN841I2ILS84,33700,CA,06099,06099000805,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,1084,2059,49,1,49.00000,Not Hispanic or Latino,Not Hispanic or Latino,White,White,Not applicable
9712774,2020,549300VZVN841I2ILS84,41740,CA,06073,06073018513,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,1849,2555,24,1,36.00000,Not Hispanic or Latino,Not Hispanic or Latino,White,White,Not applicable
9712775,2020,549300VZVN841I2ILS84,32580,TX,48215,48215024106,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,2045,3174,16,1,44.00000,Hispanic or Latino,Hispanic or Latino,White,White,Not applicable
9712776,2020,549300VZVN841I2ILS84,40140,CA,06071,06071011206,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,234,2558,43,1,25.00000,Not Hispanic or Latino,Not Hispanic or Latino,White,White,Not applicable


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9712778 entries, 0 to 9712777
Data columns (total 70 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   activity_year                      object 
 1   lei                                object 
 2   derived_msa_md                     object 
 3   state_code                         object 
 4   county_code                        object 
 5   census_tract                       object 
 6   conforming_loan_limit              object 
 7   derived_loan_product_type          object 
 8   derived_dwelling_category          object 
 9   derived_ethnicity                  object 
 10  derived_race                       object 
 11  derived_sex                        object 
 12  action_taken                       object 
 13  purchaser_type                     object 
 14  preapproval                        bool   
 15  loan_type                          object 
 16  loan_purpose      

In [4]:
# size of dataset
len(df)

9712778

## Drop rows due to inconclusive dependent variable

In [5]:
# Values:

#     1 - 1 Loan originated
#     2 - 1 Application approved but not accepted
#     3 - 0 Application denied
#     4 - *drop* Application withdrawn by applicant
#     5 - *drop* File closed for incompleteness
#     6 - 1 Purchased loan
#     7 - 0 Preapproval request denied
#     8 - 1 Preapproval request approved but not accepted

selected = df.loc[:, 'action_taken'].isin(['4', '5'])
df_selected = df[~selected]
len(df_selected)

8211413

In [6]:
# Create dependate binary variable

accepted_selected = df_selected.loc[:, 'action_taken'].isin(['1', '2', '6', '8'])
df_selected.loc[:, 'application_approved'] = 0  # Default value is 0

# Update rows where accepted_selected is True
df_selected.loc[accepted_selected, 'application_approved'] = 1

C:\Users\Forcessofnature\AppData\Local\Temp\ipykernel_10164\3826286786.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.loc[:, 'application_approved'] = 0  # Default value is 0


## Cleaning Features

In [7]:
#conbination of race and ethinicty to no leave out hispanic or latino

df_selected['race_ethnicity'] = df['derived_race'].str.cat(df['derived_ethnicity'], sep='|')

# how many categroies were created
len(df_selected.race_ethnicity.value_counts(dropna=False))

C:\Users\Forcessofnature\AppData\Local\Temp\ipykernel_10164\3418016115.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['race_ethnicity'] = df['derived_race'].str.cat(df['derived_ethnicity'], sep='|')


45

In [8]:
# Keep columns of interest
df_selected_small = df_selected.loc[:, ['application_approved', 'race_ethnicity', 'income', 'loan_amount', 'property_value', 'loan_term', 'interest_rate', 'state_code', 'activity_year', 'occupancy_type']]

In [9]:
# Remove negative income
df_selected_small = df_selected_small[df_selected_small.loc[:,'income']>0]

#Normalized Income

# Apply log transformation using numpy's log1p function
df_selected_small['income_log'] = np.log1p(df_selected_small['income'])

df_selected_small = df_selected_small[df_selected_small.loc[:,'income']>0]

# Convert the 'loan_amount' column to float64
df_selected_small['loan_amount'] = df_selected_small['loan_amount'].astype('float64')

df_selected_small['interest_rate'] = df_selected_small['interest_rate'].astype('float64')

df_selected_small['loan_term'] = df_selected_small['loan_term'].astype('float64')


df_selected_small['loan_to_value'] = df_selected_small['loan_amount'] / df_selected_small['property_value'] 

df_selected_small['debt_to_income'] = df_selected_small['loan_amount'] / df_selected_small['income'] 

df_selected_small.drop(['income'], axis=1, inplace=True) 


In [10]:
df_selected_small.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6996272 entries, 0 to 9712777
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   application_approved  int64  
 1   race_ethnicity        object 
 2   loan_amount           float64
 3   property_value        float64
 4   loan_term             float64
 5   interest_rate         float64
 6   state_code            object 
 7   activity_year         object 
 8   occupancy_type        object 
 9   income_log            float64
 10  loan_to_value         float64
 11  debt_to_income        float64
dtypes: float64(7), int64(1), object(4)
memory usage: 693.9+ MB


In [11]:

data_encoded = pd.get_dummies(df_selected_small, columns=['race_ethnicity', 'state_code', 'activity_year' ,'occupancy_type'], drop_first=True)
data_encoded


,application_approved,loan_amount,property_value,loan_term,interest_rate,income_log,loan_to_value,debt_to_income,race_ethnicity_2 or more minority races|Free Form Text Only,race_ethnicity_2 or more minority races|Hispanic or Latino,...,race_ethnicity_White|Not Hispanic or Latino,state_code_NY,state_code_TX,state_code_WA,activity_year_2019,activity_year_2020,activity_year_2021,activity_year_2022,occupancy_type_Principal Residence,occupancy_type_Second Residence
0,1,305000.0,355000.0,360.0,4.250,4.290459,0.859155,4236.111111,False,False,...,False,False,True,False,False,False,False,True,True,False
1,1,325000.0,325000.0,360.0,6.125,4.624973,1.000000,3217.821782,False,False,...,False,False,True,False,False,False,False,True,True,False
2,0,205000.0,215000.0,360.0,NaN,4.317488,0.953488,2770.270270,False,False,...,False,False,True,False,False,False,False,True,True,False
4,1,425000.0,445000.0,360.0,3.500,5.170484,0.955056,2428.571429,False,False,...,False,False,True,False,False,False,False,True,True,False
5,1,175000.0,215000.0,360.0,5.375,4.317488,0.813953,2364.864865,False,False,...,True,False,True,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9712773,1,295000.0,305000.0,360.0,2.750,3.912023,0.967213,6020.408163,False,False,...,True,False,False,False,False,True,False,False,True,False
9712774,1,585000.0,775000.0,360.0,3.500,4.948760,0.754839,4178.571429,False,False,...,True,False,False,False,False,True,False,False,True,False
9712775,1,295000.0,295000.0,360.0,3.625,4.882802,1.000000,2251.908397,False,False,...,False,False,True,False,False,True,False,False,True,False
9712776,1,215000.0,285000.0,360.0,3.875,5.587249,0.754386,808.270677,False,False,...,True,False,False,False,False,True,False,False,False,False


In [12]:
len(data_encoded.columns)

61

In [13]:
df_selected_small.to_parquet(r"C:\Users\Forcessofnature\Downloads\df_small.parquet")
data_encoded.to_parquet(r"C:\Users\Forcessofnature\Downloads\df_small_encoded.parquet")